#Configuramos las credenciales

In [0]:
%sh
mkdir /mnt/datasets

In [0]:
# Configura las credenciales de Azure
storage_account_name = "testaz2611242sa"
container_name = "datasets"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-11-26T19:41:32Z&st=2024-11-26T11:41:32Z&spr=https&sig=i36fmoRJhh9I%2BB9ZPhCGrCEgeBpU4R%2BNc%2FDVPDXBK9I%3D"

# Ruta del punto de montaje
mount_point = "/mnt/datasets"

#Montar el contenedor

In [0]:
# Desmontar si ya existe un punto de montaje en la misma ruta
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)

# Montar el contenedor
dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point = mount_point,
    extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token}
)

# Verificar el montaje
display(dbutils.fs.ls(mount_point))

/mnt/datasets has been unmounted.


path,name,size,modificationTime
dbfs:/mnt/datasets/dataset2.csv,dataset2.csv,186,1732618327000


#Read csv file from storage

In [0]:
csv = spark.read.csv("dbfs:/mnt/datasets/dataset2.csv", header=True, inferSchema=True)
display(csv)

ID,Producto,Precio,Ciudad
1,Laptop,800,Madrid
2,Smartphone,500,Barcelona
3,Tablet,250,Valencia
4,Monitor,150,Sevilla
5,Teclado,40,Bilbao
6,Auriculares,30,Valencia
7,Impresora,120,Madrid


#Read remote_table

In [0]:
remote_table = (spark.read
  .format("mysql")
  .option("dbtable", "tabla_1")
  .option("host", "testaz2611242sqldb.mysql.database.azure.com")
  .option("port", "3306") # Optional - will use default port 3306 if not specified.
  .option("database", "eoi")
  .option("user", "adminsql")
  .option("password", "Pa55w.rd1234")
  .option("useSSL", "true")
  .load()
)

display(remote_table)

id,nombre,edad,ciudad
1,Juan,25,Madrid
2,Ana,30,Barcelona
3,Luis,22,Valencia
4,Marta,28,Sevilla
5,Pedro,35,Bilbao
6,Laura,29,Valencia
7,Javier,40,Madrid
8,Cristina,27,Barcelona
9,Pablo,33,Zaragoza
10,Isabel,24,Bilbao


In [0]:
display(remote_table.join(csv, remote_table.ciudad == csv.Ciudad))

id,nombre,edad,ciudad,ID,Producto,Precio,Ciudad
1,Juan,25,Madrid,7,Impresora,120,Madrid
1,Juan,25,Madrid,1,Laptop,800,Madrid
2,Ana,30,Barcelona,2,Smartphone,500,Barcelona
3,Luis,22,Valencia,6,Auriculares,30,Valencia
3,Luis,22,Valencia,3,Tablet,250,Valencia
4,Marta,28,Sevilla,4,Monitor,150,Sevilla
5,Pedro,35,Bilbao,5,Teclado,40,Bilbao
6,Laura,29,Valencia,6,Auriculares,30,Valencia
6,Laura,29,Valencia,3,Tablet,250,Valencia
7,Javier,40,Madrid,7,Impresora,120,Madrid
